In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
import urllib.request


In [2]:
## Load the pre-trained ResNet50 model
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [3]:
# Create a custom model for binary classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 24112513 (91.98 MB)
Trainable params: 524801 (2.00 MB)
Non-trainable params: 23587712 (89.98 MB)
___________

In [4]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False


In [5]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
# Function to classify window cleanliness
def classify_window_cleanliness(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = preprocess_input(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))

    # Make predictions
    prediction = model.predict(img)
    
    # Threshold the prediction (e.g., if > 0.5, it's clean, otherwise, it's not clean)
    if prediction > 0.5:
        return "Clean"
    else:
        return "Not Clean"

In [7]:
# Example usage
image_url = 'https://live.staticflickr.com/2874/8895879575_e8cbaabdf2_b.jpg'  # Replace with the URL of your window image
image_path, _ = urllib.request.urlretrieve(image_url)
cleanliness_label = classify_window_cleanliness(image_path)
print(f"The window is classified as: {cleanliness_label}")

1/1 [==============================] - 1s 1s/step
The window is classified as: Clean
